In [1]:
import echopype as ep

from prefect import task, flow
from prefect_dask import DaskTaskRunner, get_dask_client
from distributed.utils import logging
import dask
from distributed import LocalCluster, Client

In [2]:
cluster = LocalCluster()
cluster.dashboard_link

2023-08-08 12:40:02,675 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1c/c2gm4rw56831d9cfkdf10y7h0000gn/T/dask-worker-space/worker-pledh8l2', purging
2023-08-08 12:40:02,676 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1c/c2gm4rw56831d9cfkdf10y7h0000gn/T/dask-worker-space/worker-kd_qhh7y', purging
2023-08-08 12:40:02,676 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1c/c2gm4rw56831d9cfkdf10y7h0000gn/T/dask-worker-space/worker-6vr9lqmb', purging
2023-08-08 12:40:02,677 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1c/c2gm4rw56831d9cfkdf10y7h0000gn/T/dask-worker-space/worker-be34sx1z', purging


'http://127.0.0.1:8787/status'

In [3]:
client = Client(address=cluster.scheduler_address)

In [4]:
client.scheduler

<pooled rpc to 'tcp://127.0.0.1:57028'>

In [5]:
@task
def open_raw(fpath):
    ed = ep.open_raw(fpath, sonar_model='ek60')
    print("opened raw")
    ed.to_zarr("./combined_files", overwrite=True, compute=False)
    
@flow(task_runner=DaskTaskRunner(address=cluster.scheduler_address))
def convert_files(fpaths):
    futures = [open_raw.submit(p) for p in fpaths]
    return [f.result() for f in futures]

In [6]:
import glob
fpaths = glob.glob("../notebooks/offline_files/*.raw")

In [7]:
convert_files(fpaths)

opened raw
Rechunked channel
Rechunked time1
Rechunked absorption_indicative
Rechunked sound_speed_indicative
Rechunked frequency_nominal
Rechunked latitude
Rechunked longitude
Rechunked sentence_type
Rechunked pitch
Rechunked roll
Rechunked vertical_offset
Rechunked water_level
Rechunked MRU_offset_x
Rechunked MRU_offset_y
Rechunked MRU_offset_z
Rechunked MRU_rotation_x
Rechunked MRU_rotation_y
Rechunked MRU_rotation_z
Rechunked position_offset_x
Rechunked position_offset_y
Rechunked position_offset_z
Rechunked time1
Rechunked time2
Rechunked channel
Rechunked transducer_offset_x
Rechunked transducer_offset_y
Rechunked transducer_offset_z
Rechunked frequency_nominal
opened raw
opened raw
opened raw
Rechunked channel
Rechunked time1
Rechunked absorption_indicative
Rechunked sound_speed_indicative
Rechunked frequency_nominal
Rechunked channel
Rechunked NMEA_datagramRechunked time1

opened raw
Rechunked absorption_indicative
Rechunked time1
Rechunked sound_speed_indicative
Rechunked chan

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
cluster.close()